# email draft

Hey Andres, Liu:

I finished creating a pedestrian workflow on the new codebase which can now be used to run simulations. Instructions on how to install the new code as well as how to fill the new pairing table are found here: Please read carefully before running new simulation and when updating the pairing table, please pay close attention to warnings highlighted in red, and specific keywords highlighted in grten.

Liu: 
* Please order the rwos in the pairing table such that rwos that use amenities or jobs are at the bottum of the pairing table
* please use this new updated subway file as it has an updated column for XXX


general library improvements:
* better organization of functions into folders. Better object structure that simplifies internal data storage and referencing which improves readibility and makes expansion easier. (Thanks for work done by out UROPs Kwesi and Orion)
* each version is now associated with a version number and a release date for better tracking of what version of the code was used to generate what simulation. This information is now added to the first entry of the simulation log

updates to the betweenness:
* Option to include an exposure parameters, results are reported to the origin layer
* option to expore path level data from the betweenness simulation, useful fo9r debugging and manual queries
* simplified implimintation of the betweenness that is easier to read and expand
* reduction of unnessisary loops and better use of vectorization, putential minor effeciency gains
* better and more optimized handiling of elastic weights that avoids repeated calculation. Now is a function call made inside the betweenness, leveraging internal calculations already generated

Updates to network creation:
* Significiantly faster network creation
* Significiantly Faster node insertion
* Redundanrt edges are now split into two segments, improving simulation accuricy compared to old behaviour which ignored one optipn

Updates to pairing table:
* more explicit simulation settings in the pairing table
* removal of the nested "simulation setting" folder system, and replaced with a flat folder that matches pairing table specifications. No simulation settings are needed when running a simulation as everything is explicitly set in the pairting table

Updates to logger:
* Added feature to enable storage of origin characteristics like reach, gravity and elasic weight. 
* enhancements to the flow map include showing OD snapping line
* Including the code version number and release date as part of the log
* inclufing data file projections in the log to enable quicker debugging




## TODO
# TODO
* ~~include split redundant edges into code~~
* ~~commit to github~~
* ~~update the elastic weight function to be at the origin level~~
* ~~transfer the betweenness flow with exposure into the new code~~
* ~~commit to github~~
* ~~write a documentation of the pairing table~~
* ~~update the logger and the file saving mechanisim~~
* ~~include origin and destination line connectors~~
* ~~commit to github~~
* ~~move workflow into code, create notebook of workflows~~
* ~~commit to githun~~
* ~~emale andres and lui~~

# tuesday 2023-07-11 action items:
* ~~Debug Beirut, see where the source of decreased knn weight~~
* ~~re run an accessibility simulation, as well as a betweenness flow simulation for beirut~~
* ~~Once simulation accessibility results done, send to danie~~ addd flows to folder and send another email
* ~~create visuak maos for scenarios showing increase in reach.. Add to google slides~~
* ~~Homes to > transit - parks - amenities - schools~~
* ~~geometric - no turns, no elastic weight, no decay ....~~
* ~~la_extended16_manualFixed~~
* ~~Run Somerville~~
* ~~RUn LA~~
* ~~look at Andres' email about brooklyn , run a simulation using the old code in the workstation~~
* ~~look at Lui's email, see if I need to do anything about it~~
* ~~create OD pairing table for LA, send to Andres~~
* ~~RUN A FKOW MODEL FOR HOMES TO MOSKS, USING THE SAME PARAMETERS, WITH KNN , FOR ALL SCENARIOS, SEND TO DANIEL~~
* ~~CLip lui's dataset into the area that daniel chose, share with andres~~
*~~ FOR the clip, use this file "C:\Users\abdul\Dropbox (MIT)\115_NYCWalks\03_Data\02_Edited\NYC_Sidewalk_Edit_July\BK_clipping.geojson"~~
* ~~make sure the new node insertion code works and matches well~~

## effecient node edge creation
* ~~Make sure the functionality to construct nodes-edges works for a safey buffer~~
* ~~Isolate the node insertion code into script~~
* ~~commit script to my branch~~
* ~~email orion about it, mention how easy to integrate.~~
* ~~look into issue with redundant edges~~

## Testing
* ~~structure tests so its a one csv: first n rows are settings, next m rows are edges flows, with first column being same segment ids from network file~~
* ~~standard test case is four files: network.jeojson - origins.geojson - destination.jeojson - testflows.csv~~
* ~~function reads a csv into list of dicts: test settings and series of output.~~
* ~~build Harvard Square testflows.csv~~
~~* make sure this config runs on Harvard Square~~



# TODO
## Functionality List 1
* add visualization functionality to new version of Madina so testing is easier
* in network, add  connector segments as a separate layer (Maybe not very needed if origins and destinations visually are snapoped...)
* Commit visualization updates to github


## Testing
* digitize three test cases for manhattan, visualize OD snapping, show where differences (if any) come from between Rhino  and Python in the NYC case.
* Put maps into slides
* CHeck somerville, to see if we delete the same number of edges, with and without the effecient code "C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\Cities\Somerville\Data\Somerville_network_1000m_buffer.geojson"

## Functionality List 2
* build the betweenness simulation workflow, combine with accissibility
* include Logger as a class, and zonal would have an instance of that class. Logger handles event documentation, and captures output.
* for each origin, have an origin record, showing its knn weight, reach, gravity towards each of its destination, header rows showing settings and parameters
* for the network, columns of od flows, headed  by settings and parameters rows that explicitly detail units, weight type, calibration status, .. any relevant data, settings and parameters..
* set up a slideshow/diagram to show the relationship between different compoonents of the library.
* Set up a notebook to go over estimating flows from one origin to one destination, Document all relevant settings
* show how this generalizes to the pairings.csv, iterating over moodel settings, iterating over scenarios
* Make a finalized github commit including testing script, betweenness flow notebook. 
* Deploy to workstation.


## Functionality List 3
* given the simulation results, and the censor locations, find a way to calibrate counts and generate beta coeffecients (Beirut, SOmerville, NYC)
* enable Jupyter notebook server access from workstation.
* Figure out a way to do the "Cities" dropbox folder, people can add data through dropbox, run simulations through Jupyter Server, get results through Dropbox




## Running Cases
* Send output of LA to Nico when done
* send Brooklyn output when done
* be prepared to run entire NYC simulation, be mindful that daniel already sent more network parts


## Map Cleaning
https://www.dropbox.com/scl/fo/d6ugcyt3aanfb3yk4n9va/h?dl=0&rlkey=1c2giilvzoyygcxrlry7fscr9



! pip install tqdm
#from tqdm import tqdm
import sys
print(sys.version)

In [ ]:
list(test_config.index[0:2])

In [1]:
import sys
sys.path.append('../')



from madina.zonal.zonal import Zonal
from madina.una.betweenness import parallel_betweenness
from madina.una.elastic import get_elastic_weight
from madina.una.betweenness import paralell_betweenness_exposure

import os
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
import numpy as np

from geopandas import GeoDataFrame, GeoSeries


print (f"{'test name':30s} | {'madina_flow_sum':15s} | {'Rhino_flow_sum':15s} | {'sum_diff':8s} | {'sum_smlr_%':10s} | {'sle_mean':8s} | {'sle_median':10s} | {'sle_max':8s} | {'sle>0.1%':8s} | {'sle>1.0%':8s} | {'sle>3.0%':8s} | {'sle>5.0%':8s}")
#for test_case in os.listdir("Test Cases"):
for test_case in ['Somerville']:   # 'Harvard Square',
    # TODO: Check OS compatibility, ensure this is compatible with Unix systems..
    test_case_folder = "Test Cases" + "\\" + test_case + "\\"
    test_config = pd.read_csv(test_case_folder + "test_configs.csv")
    test_flows =  pd.read_csv(test_case_folder + "test_flows.csv")

    if test_case == 'Somerville':
        ready_tests = test_config.index[0:2]
    else:
        ready_tests = test_config.index

    for test_idx in ready_tests:

        harvard_square = Zonal()

        harvard_square.load_layer(
            layer_name='streets',
            file_path=  test_case_folder + test_config.at[test_idx, 'Network_File']
            )

        harvard_square.load_layer(
            layer_name=test_config.at[test_idx, 'Origin_Name'],
            file_path= test_case_folder + test_config.at[test_idx, 'Origin_File']
            )

        harvard_square.load_layer(
            layer_name=test_config.at[test_idx, 'Destination_Name'],
            file_path= test_case_folder + test_config.at[test_idx, 'Destination_File']
            )
        

        harvard_square.create_street_network(
            source_layer='streets', 
            discard_redundant_edges=False,
            node_snapping_tolerance=0.01,  # TODO: check for sensitivity... pick one as default snapping.
            weight_attribute=test_config.at[test_idx, 'Network_Cost'] if test_config.at[test_idx, 'Network_Cost'] != "Geometric" else None
        )

        #print (f"{harvard_square.network.edges.shape = }")
        #print (f"{harvard_square.network.nodes.shape = }")
        #harvard_square.network.nodes,  harvard_square.network.edges = _discard_redundant_edges(harvard_square.network.nodes, harvard_square.network.edges)
        #print (f"{harvard_square.network.edges.shape = }")
        #print (f"{harvard_square.network.nodes.shape = }")

        harvard_square.insert_node(
            layer_name=test_config.at[test_idx, 'Origin_Name'], 
            label='origin', 
            weight_attribute=test_config.at[test_idx, 'Origin_Weight'] if test_config.at[test_idx, 'Origin_Weight'] != "Count" else None
        )
        harvard_square.insert_node(
            layer_name=test_config.at[test_idx, 'Destination_Name'], 
            label='destination', 
            weight_attribute=test_config.at[test_idx, 'Destination_Weight'] if test_config.at[test_idx, 'Destination_Weight'] != "Count" else None
        )

        harvard_square.create_graph()

        node_gdf = harvard_square.network.nodes
        origin_gdf = node_gdf[node_gdf['type'] == 'origin']

        harvard_square.network.nodes["original_weight"] = harvard_square.network.nodes["weight"]

        harvard_square.network.turn_penalty_amount = test_config.at[test_idx, 'Turn_Penalty']
        harvard_square.network.turn_threshold_degree = test_config.at[test_idx, 'Turn_Threshold']

        if test_config.at[test_idx, 'Elastic_Weights']:
            '''
            harvard_square.network.nodes["weight"] = harvard_square.network.nodes["original_weight"]
            get_elastic_weight(
                harvard_square.network,
                search_radius=test_config.at[test_idx, 'Radius'],
                detour_ratio=test_config.at[test_idx, 'Detour'],
                beta=test_config.at[test_idx, 'Beta'],
                decay=True, #test_config.at[test_idx, 'Decay'],
                #turn_penalty=test_config.at[test_idx, 'Turns'],
                turn_penalty=False,
            )
            for o_idx in origin_gdf.index:
                harvard_square.network.nodes.at[o_idx, 'weight'] =  harvard_square.network.nodes.at[o_idx, 'elastic_weight']
            '''
            continue
        '''
        return_dict = parallel_betweenness(
            harvard_square.network,
            search_radius=test_config.at[test_idx, 'Radius'],
            detour_ratio=test_config.at[test_idx, 'Detour'],
            decay=test_config.at[test_idx, 'Decay'], #if test['Elastic weights'] else True,
            decay_method=test_config.at[test_idx, 'Decay_Mode'],  # "power", "exponent"
            beta=test_config.at[test_idx, 'Beta'],
            path_detour_penalty='equal', # "equal",  # "power", "exponent", "equal"
            origin_weights=False if type(test_config.at[test_idx, 'Origin_Weight']) != str else True,
            closest_destination=test_config.at[test_idx, 'Closest_Destination'],
            destination_weights=False if type(test_config.at[test_idx, 'Destination_Weight']) != str  else True,    #or (test['Elastic weights'])
            # perceived_distance=False,
            num_cores=6,
            light_graph=True,
            turn_penalty=test_config.at[test_idx, 'Turns'],
        )
        
        '''
        return_dict = paralell_betweenness_exposure(
            harvard_square,
            search_radius=test_config.at[test_idx,'Radius'],
            detour_ratio=test_config.at[test_idx,'Detour'],
            decay=False if test_config.at[test_idx,'Elastic_Weights'] else test_config.at[test_idx,'Decay'],  # elastic weight already reduces origin weight factoring in decay. if this pairing uses elastic weights, don't decay again,
            decay_method=test_config.at[test_idx,'Decay_Mode'],
            beta=test_config.at[test_idx,'Beta'],
            num_cores=6,
            path_detour_penalty='equal', # "power" | "exponent" | "equal"
            closest_destination=test_config.at[test_idx,'Closest_Destination'],
            elastic_weight=test_config.at[test_idx,'Elastic_Weights'],
            turn_penalty=test_config.at[test_idx,'Turns'],
            path_exposure_attribute=None,
            return_path_record=False, 
            destniation_cap=None
        )
        



        simulated_sum_of_flow = return_dict['edge_gdf']['betweenness'].sum()
        test_flow = test_flows[test_config.at[test_idx, 'Flow_Name']].sum()


        ## create segment level comparison
        # creating connector lines

        import shapely.geometry as geo
        simulated_betweenness = return_dict['edge_gdf'][['betweenness', 'parent_street_id']].rename(columns={"betweenness": "simulated_betweenness"}).drop_duplicates(subset=['parent_street_id']).set_index("parent_street_id")
        simulated_betweenness = harvard_square.layers["streets"].gdf[["geometry", "__GUID"]].join(simulated_betweenness).set_index("__GUID")

        test_name = test_config.at[test_idx, 'Flow_Name']
        test_betweenness = test_flows[['__GUID', test_name]].set_index("__GUID").rename(columns = {test_name: "test_flow"})


        comparison = simulated_betweenness.join(test_betweenness)
        comparison["difference"] = comparison["simulated_betweenness"] - comparison["test_flow"]
        comparison["difference_pct"] = abs(comparison["difference"]) / comparison["simulated_betweenness"] *100
        # segment level error
        sle = comparison[~comparison["difference_pct"].isna()]['difference_pct']

        #'''
        node_gdf = harvard_square.network.nodes
        edge_gdf = harvard_square.network.edges


        origin_layer = harvard_square.layers[test_config.at[test_idx, 'Origin_Name']].gdf
        origin_nodes = node_gdf[node_gdf['type'] == 'origin']
        origin_joined = origin_layer.join(origin_nodes.set_index('source_id'),lsuffix='_origin')
        origin_joined['connector_line'] = origin_joined.apply(lambda x:geo.LineString([x['geometry'], x["geometry_origin"]]), axis=1)
        origin_joined["geometry"] = origin_joined['connector_line']


        destination_layer = harvard_square.layers[test_config.at[test_idx, 'Destination_Name']].gdf
        destination_nodes = node_gdf[node_gdf['type'] == 'destination']
        destination_joined = destination_layer.join(destination_nodes.set_index('source_id'),lsuffix='_destination')
        destination_joined['connector_line'] = destination_joined.apply(lambda x:geo.LineString([x['geometry'], x["geometry_destination"]]), axis=1)
        destination_joined["geometry"] = destination_joined['connector_line']

        streets = harvard_square.layers["streets"].gdf 
        network_file = gpd.read_file(test_case_folder + test_config.at[test_idx, 'Network_File'], engine='pyogrio')


        flow_difference = comparison[
            ((comparison['test_flow' ] > 0) & (comparison['simulated_betweenness'] == 0)) | 
            ((comparison['test_flow' ] == 0) & (comparison['simulated_betweenness'] > 0))
        ]
        harvard_square.create_map(
            [
                #{'gdf': streets[streets[test_config.at[test_idx, 'Network_Cost']] > 0], 'color': [255, 255, 255], 'text': test_config.at[test_idx, 'Network_Cost']},
                {'gdf': streets, 'color': [100, 100, 100], 'opacity': 0.1},
                {'gdf': edge_gdf[edge_gdf['betweenness'] > 0], 'color': ['125, 125, 0'], 'text': 'betweenness', 'opacity': 0.2},
                #{'gdf': comparison[abs(comparison["difference"]) > 0.01], 'color_by_attribute': 'difference', 'color_method': 'gradient', 'text': 'difference'},
                {'gdf': comparison[(comparison["difference_pct"] >= 0.1) & (comparison["difference_pct"] < 100)], 'color_by_attribute': 'difference_pct', 'color_method': 'gradient', 'text': 'difference_pct'},
                {'gdf': edge_gdf[edge_gdf['snapped'] == True], 'color': [255, 0, 255], 'opacity': 0.2},
                {'gdf': network_file[network_file["geometry"].geom_type == 'Polygon'], 'color': [125, 0, 125], 'text': '__GUID'},
                {'gdf': flow_difference, 'color': [255, 255, 0] , 'text': 'difference'},        
                #{'gdf': comparison[comparison['difference'] != 0], 'color_by_attribute': 'difference', 'color_method': 'gradient', 'text': 'difference', 'opacity':  0.1},
                {'gdf': origin_layer, 'color': [0, 0, 255]},
                {'gdf': origin_joined[['geometry']], 'color': [0, 0, 255]},
                {'gdf': destination_layer, 'color': [255, 0, 0]},
                {'gdf': destination_joined[['geometry']], 'color': [255, 0, 0]},
                {'gdf': harvard_square.network.nodes[['geometry', 'type', 'weight']].reset_index(), 'color': [255, 0, 255], 'text': 'id'},
            ],
            save_as="Test Cases\\" + test_case + '\\'  + test_name + "._difference_map.html"
        )
        #'''
        #print (test_config.loc[test_idx])
        print (f"{test_config.at[test_idx, 'Flow_Name'][:30]:30s} | {simulated_sum_of_flow:15.2f} | {test_flow:15.2f} | {simulated_sum_of_flow - test_flow:8.2f} | {1-(simulated_sum_of_flow - test_flow)/ test_flow:10.2%} | {sle.mean():7.4f}% | {sle.median():9.4f}% | {sle.max():7.4f}% | {sle[sle > 0.1].count():8} | {sle[sle > 1.0].count():8} | {sle[sle > 3.0].count():8} | {sle[sle > 5.0].count():8}")
        #print ("DOne Case...")
        #break

c:\Users\abdul\.conda\envs\madina_dev\Lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
c:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\..\madina\zonal\network.py:6: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shap

test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0%
Somerville_Bus_Subway_Geometri |          700.29 |          713.14 |   -12.84 |    101.80% |  2.3488% |    2.2893% |  5.0852% |      266 |      232 |       96 |        2
Somerville_Bus_Subway          |          730.95 |          743.95 |   -13.00 |    101.75% |  2.4117% |    2.2917% |  5.1912% |      285 |      251 |      103 |        8


In [ ]:
### paralell_betweenness_exposure
test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0%
Somerville_Bus_Subway_Geometri |          700.29 |          713.14 |   -12.84 |    101.80% |  2.3488% |    2.2893% |  5.0852% |      266 |      232 |       96 |        2
Somerville_Bus_Subway          |          730.95 |          743.95 |   -13.00 |    101.75% |  2.4117% |    2.2917% |  5.1912% |      285 |      251 |      103 |        8
### parallel_betweenness



### parallel_betweenness
test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0%
_Betweenness1                  |          704.85 |          702.07 |     2.78 |     99.60% |  0.0376% |    0.0000% |  2.4893% |        2 |        2 |        0 |        0
_Betweenness2                  |        19116.27 |        19025.63 |    90.64 |     99.52% |  0.0439% |    0.0000% |  3.1837% |        2 |        2 |        1 |        0
_Betweenness3                  |        19299.00 |        19214.00 |    85.00 |     99.56% |  0.0295% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0
_Betweenness4                  |        20938.18 |        20853.18 |    85.00 |     99.59% |  0.0206% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0
Somerville_Bus_Subway_Geometri |          709.44 |          713.14 |    -3.69 |    100.52% |  0.5027% |    0.1954% |  3.9604% |      149 |       62 |        3 |        0
Somerville_Bus_Subway          |          740.19 |          743.95 |    -3.76 |    100.51% |  0.5074% |    0.2539% |  3.9604% |      173 |       39 |        3 |        0
Somerville_Homes_Subway        |       377922.37 |       389383.31 | -11460.94 |    102.94% |     inf% |    3.8409% |     inf% |     2578 |     2232 |     1591 |      997
Somerville_Jobs_Subway         |       146309.13 |       149908.30 | -3599.17 |    102.40% |     inf% |    3.5776% |     inf% |     2078 |     1741 |     1286 |      824

# COnstructing the Manhattan Case..
## Don't forget to check for node insertion, set  return_all=False in the spatial index

In [ ]:
import time
start = time.time()

buildings_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\Home_PT_6538.geojson"
subway_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\Metro_PT_6538.geojson"
network_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\network_clipped_dupremovedAS.geojson"

import sys
sys.path.append('../')
from madina.zonal.zonal import Zonal


harvard_square = Zonal()

print(f"{(time.time()-start)*1000:6.2f}ms\t imports done, object created")
start = time.time()

#harvard_square.load_layer(
#    layer_name='streets',
#    file_path=network_file
#    )


print(f"{(time.time()-start)*1000:6.2f}ms\t street data loaded")
start = time.time()

harvard_square.load_layer(
    layer_name='buildings',
    file_path=buildings_file
    )

buildings_gdf = harvard_square.layers['buildings'].gdf
harvard_square.layers['buildings'].gdf = buildings_gdf[~buildings_gdf['FID'].isin([27967, 9140, 3974])]

print(f"{(time.time()-start)*1000:6.2f}ms\t building data loaded")
start = time.time()

harvard_square.load_layer(
    layer_name='subway',
    file_path=subway_file
    )

print(f"{(time.time()-start)*1000:6.2f}ms\t subway data loaded")

In [ ]:
start = time.time()

harvard_square.create_street_network(
    source_layer='streets', 
    discard_redundant_edges=False, 
    node_snapping_tolerance=0
)

print(f"{(time.time()-start)*1000:6.2f}ms\t street network created")
start = time.time()

harvard_square.insert_node(
    layer_name='buildings', 
    label='origin', 
    weight_attribute='TotalPop'
)


print(f"{(time.time()-start)*1000:6.2f}ms\t origins insertes")
start = time.time()

harvard_square.insert_node(
    layer_name='subway', 
    label='destination', 
    weight_attribute='line_ent_st'
)

print(f"{(time.time()-start)*1000:6.2f}ms\t destinations insertes")


start = time.time()

harvard_square.create_graph(light_graph=True, d_graph=True)

print(f"{(time.time()-start)*1000:6.2f}ms\t graph created insertes")
start = time.time()

return_dict = parallel_betweenness(
    harvard_square.network,
    search_radius=800,
    detour_ratio=1.15,
    decay=False,
    decay_method='exponent',  # "power", "exponent"
    beta=0.004,
    path_detour_penalty="equal",  # "power", "exponent", "equal"
    origin_weights=True,
    closest_destination=False,
    destination_weights=True, 
    # perceived_distance=False,
    num_cores=8,
    light_graph=True,
    turn_penalty=False,
)
simulated_sum_of_flow = return_dict['edge_gdf']['betweenness'].sum()

print(f"{(time.time()-start)*1000:6.2f}ms\t Betweenness estimated")
start = time.time()

In [ ]:
joined_results = harvard_square.layers['streets'].gdf.join(harvard_square.network.edges[['parent_street_id', 'betweenness']].set_index('parent_street_id'))
joined_results[['__GUID', 'betweenness', 'geometry']].to_csv('2023-07-07 manhattan betweenness flow test.csv')
joined_results[['__GUID', 'betweenness', 'geometry']].to_file('2023-07-07 manhattan betweenness flow test.geoJSON', driver="GeoJSON")

In [ ]:
harvard_square.create_map(
    [
        #{
            #'layer': 'streets',
            #'color': [125, 125, 125],
        #},
        {
            'gdf': harvard_square.layers['homes'].gdf,
            'color_by_attribute': 'line_ent_st',
            "color_method": 'gradient'
        }
    ],
    save_as="map.html"
)

## Need to instal pydeck jupyter nb extension

`jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck`

`jupyter nbextension enable --sys-prefix --py pydeck`
